## Class which replicates the molecular optimisation workflow 

In [4]:
class workflow():
    
    def __init__(self,dataset,molecule,name):
        self.dataset = client.get_collection('OptimizationDataset','project_molecules')
        self.molecule=molecule
        self.xyz_file = f'/home/fastdata/Shared/Masters_Project/Molecules/{molecule}.xyz' # as in the command line
        self.name = name
        
    def create_molecule(self):
        with open(self.xyz_file,'r') as file:
            lines=file.readlines()[2:]
            lines=[' '.join(line.split()) for line in lines] # clean each line by stripping extra speaces and newlines and join them with a single space   
            lines = ''+ '\n'.join(lines)+''
            lines=str(lines)
        mol = ptl.Molecule.from_data(lines)
        return mol
        
    def add_specification(self):
        self.specification={'name':self.name, 
        'description':f'Geometric +Psi4/{self.name.split("_")[0]}/{self.name.split("_")[1]}.',
        'optimization_spec':{'program':'geometric','keywords':None},
        'qc_spec':{'driver':'gradient',
        'method':self.name.split('_')[0],
        'basis':self.name.split('_')[1],
        'keywords':None,
        'program': 'psi4'}}
            
        return self.specification
        
        
    def setup_molecule(self):
        #deleting the molecule if it exists
        if self.molecule.lower() in self.dataset.data.records:
            del self.dataset.data.records[f"{self.molecule.lower()}"]
            self.dataset.save()
            
        self.dataset.add_entry(f"{self.molecule}",
                                workflow(self.dataset,self.molecule,self.name).create_molecule())
            
        self.dataset.save()
        #adding the specification if not exists
        if workflow(self.dataset,self.molecule,self.name).add_specification()['name'].lower() not in self.dataset.data.specs:
            self.dataset.add_specification(**workflow(self.dataset,self.molecule,self.name).add_specification(),overwrite=True)
            self.dataset.save()
        status=self.dataset.status(collapse=False)
        return status
            
    def compute_molecule(self,tag):
        
        self.dataset.compute(workflow(self.dataset,self.molecule,self.name).add_specification()['name'],tag=tag,
                            subset={f"{self.molecule}"})
        self.dataset=client.get_collection('OptimizationDataset','project_molecules')
        status=self.dataset.status(collapse=False)
            
        return status
        
    def check_status_only(self):
        return self.dataset.status(collapse=False)
            
    def get_results(self):
        mol_results= self.dataset.get_record(f'{self.molecule}',workflow(self.dataset,self.molecule,self.name).add_specification()['name'])
        mol_results_dict = mol_results.dict()
        return mol_results_dict,mol_results
    
    def restart_job(self,new_tag):
        #new_tag = "m_project"

    #for spec in ds_mols.df.columns:
     #   for entry in ds_mols.df.index:
        rr = self.dataset.get_record(self.molecule,workflow(self.dataset,self.molecule,self.name).add_specification()['name'])
        ct =  client.modify_tasks("modify", new_tag=new_tag, base_result=rr.id)
        cr =  client.modify_tasks("restart", base_result=rr.id)
        return self.molecule,
        workflow(self.dataset,self.molecule,self.name).add_specification()['name']
        ,ct,cr
            

## Code which gets all of the function methods for a collection (Note: collection name needs to be called first)

In [1]:
import inspect

methods_and_docstrings = {
    name: obj.__doc__ for name, obj in inspect.getmembers(ds_be) 
    if callable(obj) and not name.startswith("__")
}

for method, doc in methods_and_docstrings.items():
    print(f"Method: {method}")
    print(f"Description: {doc}")
    print("-" * 40)

NameError: name 'ds_be' is not defined

## Nifty Code to get the molecular coordinates (x,y,z for each atom) in a list format from pretty_print

In [ ]:
a=(r.get_final_molecule().pretty_print()).split('\n') # Split by new line
    data = [s.split() for s in a # splits by whitespace for each line and obtains the numberical values in each line
         if  s.strip() and not s.startswith(('Geometry', 'Center', '-'))][3:]
    cleaned_data=[[d[0]]+ list(map(float,d[1:]))for d in data]